In [1]:
# !pip install h3
# !pip install plotly
# !pip install haversine
# !pip install pyarrow

In [2]:
import h3
import numpy as np
import pandas as pd
import plotly.express as px
import scipy.sparse as sparse
from haversine import haversine
import matplotlib.pyplot as plt
from datetime import datetime, timezone

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_colwidth', 1000)

In [3]:
checkins = pd.read_csv('checkins.csv')
estado = 'New_York'

In [4]:
checkins['local_datetime'] = pd.to_datetime(checkins['local_datetime'])
checkins = checkins.rename(columns={'userid': 'user_id'})

In [5]:
checkins['user_id'].nunique()

322

# Region View

In [6]:
def generate_h3_cell(row):
    lat = row['latitude']
    lon = row['longitude']
    resolution = 9

    h3_cell = h3.latlng_to_cell(lat, lon, resolution)
    return h3_cell

In [7]:
checkins['region_id'] = (
    checkins
    .apply(generate_h3_cell, axis=1)
)

checkins['region_id'].nunique()

2175

## Region Embeddings

In [8]:
dimension = 10
embeddings_df = pd.read_csv(f'{estado}_embeddings_dimension_{dimension}.csv')

In [9]:
def criar_dicionario(n):
  """
  Cria um dicionário com n chaves e valores.

  Argumentos:
    n: Número inteiro que define o número de chaves e valores.

  Retorno:
    Dicionário com chaves de '0' a 'n-1' e valores de 'feature_1' a 'feature_n-1'.
  """
  dicionario = {}
  for i in range(n):
    chave = str(i)
    valor = "feature_" + str(i + 1)
    dicionario[chave] = valor
  return dicionario

In [10]:
rename_columns = criar_dicionario(dimension)

checkins_embeddings_df = (
    checkins
    .merge(embeddings_df, on='region_id')
    .rename(columns=rename_columns)
)

In [11]:
checkins_embeddings_df['user_id'].nunique()

296

In [12]:
qt_lugares_por_usuario = (
    checkins_embeddings_df
    .groupby('user_id')
    .agg(
        {'placeid':'nunique'}
    )
    .sort_values(by=['placeid'], ascending=False)
    .reset_index()
)

# filtrando usuarios com menos de 30 checkins
qt_lugares_por_usuario = qt_lugares_por_usuario[qt_lugares_por_usuario['placeid'] >= 2]

usuarios_validos = qt_lugares_por_usuario['user_id'].unique()

checkins_embeddings_df = checkins_embeddings_df[checkins_embeddings_df['user_id'].isin(usuarios_validos)]

### Generate user regions embeddings

In [13]:
import json

def generate_user_embeddings(checkins_embeddings_df: pd.DataFrame) -> pd.DataFrame:
    users = checkins_embeddings_df['user_id'].unique()
    user_embeddings = pd.DataFrame(columns=['user_id', 'embeddings'])

    for user in users:
        user_checkins = checkins_embeddings_df[checkins_embeddings_df['user_id'] == user]
        feature_cols = (
            checkins_embeddings_df
            .filter(like='feature')
            .columns
            .tolist()
        )
        user_embedding = user_checkins[feature_cols].values
        user_embedding = json.dumps(user_embedding.tolist())

        user_embeddings = pd.concat(
            [user_embeddings, pd.DataFrame({'user_id': user, 'embeddings': [user_embedding]})], 
            ignore_index=True
        )
    
    return user_embeddings

In [14]:
checkins_embeddings_df = checkins_embeddings_df.sort_values(by=['user_id'])
user_embeddings_df = generate_user_embeddings(checkins_embeddings_df)

In [15]:
(
    user_embeddings_df
    .to_csv(f'{estado}_user_embeddings_dimension_{dimension}.csv', index=False, sep=',')
)

In [16]:
(
    checkins_embeddings_df
    .drop(columns=['region_id'] + list(rename_columns.values()))
    .rename(columns={'user_id': 'userid'})
    .to_csv(f'{estado}_checkins_filtered.csv', index=False, sep=',')
)

In [17]:
# (
#     checkins
#     .groupby('user_id')
#     .agg(
#         qt_regioes_visitadas=('h3_cell', 'nunique'), 
#         frequencia_visitas_regioes=('h3_cell', 'count'), 
#         qt_localizacoes_visitadas=('placeid', 'nunique')
#     )
#     .sort_values(by=['qt_regioes_visitadas'], ascending=False)
# )

In [18]:
# px.box(
#     (
#         checkins
#         .groupby('user_id')
#         .agg(
#             qt_regioes_visitadas=('h3_cell', 'nunique'), 
#             frequencia_visitas_regioes=('h3_cell', 'count'), 
#             qt_localizacoes_visitadas=('placeid', 'nunique')
#         )
#         .sort_values(by=['qt_regioes_visitadas'], ascending=False)
#     ),
#     y='qt_regioes_visitadas'
# )

## Region - region (distance)

**Entrada:**
Check-ins com **região h3**

**Saída:**
**Matriz de features** entre as regiões n x n, no qual **n** representa o número de regiões e cada elemento **i j** representa **a distância da região i até a região j em metros**.

É necessário também pegar o de-para das categorias que o Cláudio usa, de forma a usar um valor inteiro, ao invés do nome da categoria

In [19]:
# def h3_centroid_distance(cell_h3_1, cell_h3_2):
#     centroid_1 = h3.h3_to_geo(cell_h3_1)
#     centroid_2 = h3.h3_to_geo(cell_h3_2)
    
#     distance_meters = haversine(centroid_1, centroid_2) * 1000
#     return distance_meters

In [20]:
# def generate_user_region_distance_matrix(checkins):
#     users = checkins['user_id'].unique()

#     users_regions_distance = pd.DataFrame(columns=['user_id', 'matrices'])
#     all_regions = checkins['h3_cell'].unique().tolist()
#     all_regions_set = set(all_regions)

#     for user in users:
#         user_checkins = checkins[checkins['user_id'] == user].sort_values(by='local_datetime')

#         user_h3_regions = user_checkins['h3_cell'].unique().tolist()

#         # calcula k-hop de cada região
#         k = 3   # k-hop neighborhood

#         user_h3_k_hop_regions = user_h3_regions.copy()

#         for region in user_h3_regions:
#             neighbors_regions_set = set(list(h3.k_ring(region, k)))

#             intersecao = list(all_regions_set & neighbors_regions_set)

#             user_h3_k_hop_regions.extend(intersecao)

#         user_h3_k_hop_regions = list(set(user_h3_k_hop_regions))
        
#         # cria mapeamento de regiões para construir a matriz
#         regions_map = []
#         i = 0
#         for h3_region in user_h3_k_hop_regions:
#             region_map = {'region_id': i, 'value': h3_region}
#             i = i+1
#             regions_map.append(region_map)

#         region_distance_matrix = []

#         # calcula distância entre regiões
#         for region_map_i in regions_map:
#             region_h3_i = region_map_i['value']
#             distances_i_j = []

#             for region_map_j in regions_map:
#                 region_h3_j = region_map_j['value']
                
#                 distance = round(h3_centroid_distance(region_h3_i, region_h3_j), 2)
#                 distances_i_j.append(distance)
            
#             region_distance_matrix.append(distances_i_j)

#         novo_dado = {
#             'user_id': user, 
#             'matrices': str(region_distance_matrix),
#             # 'category': str(user_checkins['category'].unique()) 
#         }

#         aux_df = pd.DataFrame(novo_dado, index=[0])

#         users_regions_distance = pd.concat([users_regions_distance, aux_df], ignore_index=True)

#     return users_regions_distance

In [21]:
# users_regions_distance = generate_user_region_distance_matrix(checkins)

In [22]:
# users_regions_distance.head(3)

In [23]:
# users_regions_distance.shape

In [24]:
# users_regions_distance[['user_id', 'matrices']].to_csv(f'region_distance_feature_{estado}.csv', index=False)

## Region - region (adjacency matrix)

**Entrada:**
Check-ins com **região h3**

**Saída:**
**Matriz de adjacência** entre as regiões n x n, no qual **n** representa o número de regiões e cada elemento **i j** representa se **a região j sucede a região i, de acordo com a data** do checkin. A informação em i e j é ponderada, **indicando o número de vezes que j sucede i**.

In [25]:
# def generate_user_region_adjacency_matrix(checkins):
#     users = checkins['user_id'].unique()

#     users_regions_adjacency = pd.DataFrame(columns=['user_id', 'matrices'])
#     all_regions = checkins['h3_cell'].unique().tolist()
#     all_regions_set = set(all_regions)

#     for user in users:
#         user_checkins = checkins[checkins['user_id'] == user]
#         user_checkins = user_checkins.sort_values(by='local_datetime')
#         user_h3_regions = user_checkins['h3_cell'].unique().tolist()

#         # calcula k-hop de cada região
#         k = 3   # k-hop neighborhood

#         user_h3_k_hop_regions = user_h3_regions.copy()

#         for region in user_h3_regions:
#             neighbors_regions_set = set(list(h3.k_ring(region, k)))

#             intersecao = list(all_regions_set & neighbors_regions_set)

#             user_h3_k_hop_regions.extend(intersecao)

#         user_h3_k_hop_regions = list(set(user_h3_k_hop_regions))

#         adjacency_matrix = pd.DataFrame(0, index=user_h3_k_hop_regions, columns=user_h3_k_hop_regions)

#         for i in range(len(user_checkins) - 1):
#             localizacao_atual = user_checkins.iloc[i]['h3_cell']
#             localizacao_proxima = user_checkins.iloc[i + 1]['h3_cell']
#             adjacency_matrix.at[localizacao_atual, localizacao_proxima] += 1

#         novo_dado = {
#             'user_id': user, 
#             'matrices': str(adjacency_matrix.values.tolist()),
#         }

#         aux_df = pd.DataFrame(novo_dado, index=[0])

#         users_regions_adjacency = pd.concat([users_regions_adjacency, aux_df], ignore_index=True)   

#     return users_regions_adjacency

In [26]:
# users_regions_adjacency_matrix = generate_user_region_adjacency_matrix(checkins)

In [27]:
# users_regions_adjacency_matrix.shape

In [28]:
# users_regions_adjacency_matrix[['user_id', 'matrices']].to_csv(f'region_adjacency_matrix_{estado}.csv', index=False)

## Region - region (adjacency feature)

**Entrada:**
Check-ins com **região h3**

**Saída:**
**Matriz de feature** entre as regiões n x n, no qual **n** representa o número de regiões e cada elemento **i j** representa se a região i é adjacente a região j, geograficamente falando.

In [29]:
# def generate_user_region_adjacency_feature(checkins):
#     users = checkins['user_id'].unique()

#     users_regions_adjacency_feature = pd.DataFrame(columns=['user_id', 'matrices'])
#     all_regions = checkins['h3_cell'].unique().tolist()
#     all_regions_set = set(all_regions)

#     for user in users:
#         user_checkins = checkins[checkins['user_id'] == user].sort_values(by='local_datetime')
#         user_h3_regions = user_checkins['h3_cell'].unique().tolist()

#         # calcula k-hop de cada região
#         k = 3   # k-hop neighborhood

#         user_h3_k_hop_regions = user_h3_regions.copy()

#         for region in user_h3_regions:
#             neighbors_regions_set = set(list(h3.k_ring(region, k)))

#             intersecao = list(all_regions_set & neighbors_regions_set)

#             user_h3_k_hop_regions.extend(intersecao)

#         user_h3_k_hop_regions = list(set(user_h3_k_hop_regions))
        
#         # cria mapeamento de regiões para construir a matriz
#         regions_map = []

#         i = 0
#         for h3_region in user_h3_k_hop_regions:
#             region_map = {'region_id': i, 'value': h3_region}
#             i = i+1
#             regions_map.append(region_map)

#         adjacency_matrix = []

#         for region_map_i in regions_map:
#             region_h3_i = region_map_i['value']
#             adjacency_i_j = []

#             for region_map_j in regions_map:
#                 region_h3_j = region_map_j['value']   

#                 if (h3.h3_indexes_are_neighbors(region_h3_i, region_h3_j)):
#                     value = 1
#                 else:
#                     value = 0

#                 adjacency_i_j.append(value)
            
#             adjacency_matrix.append(adjacency_i_j)
        
#         novo_dado = {
#             'user_id': user, 
#             'matrices': str(adjacency_matrix),
#             # 'category': str(user_checkins['category'].unique()) 
#         }

#         aux_df = pd.DataFrame(novo_dado, index=[0])

#         users_regions_adjacency_feature = pd.concat([users_regions_adjacency_feature, aux_df], ignore_index=True)

#     return users_regions_adjacency_feature

In [30]:
# users_regions_adjacency_feature = generate_user_region_adjacency_feature(checkins)

In [31]:
# users_regions_adjacency_feature.shape

In [32]:
# users_regions_adjacency_feature[['user_id', 'matrices']].to_csv(f'region_adjacency_feature_{estado}.csv', index=False)

## Region - POI (checkins quantity)

**Entrada:**

Checkins com uma **região h3** associada à latitude e longitude 

**Saída:**
**Matriz de features **contendo em cada linha** a região h3** e em cada coluna a **quantidade de check-ins em um dado POI**.

In [33]:
# feature_matrix_poi_region_qt = (
#     pd.crosstab(
#         index=checkins_filtrados['h3_cell'], columns=checkins_filtrados['category']
#     )
# )

# feature_matrix_poi_region_qt.columns = (
#     [f'{col}' for col in feature_matrix_poi_region_qt.columns]
# )

# feature_matrix_poi_region_qt = feature_matrix_poi_region_qt.sort_index()

In [34]:
# feature_matrix_poi_region_qt

## Region - POI (poi quantity)

**Entrada:**

Checkins com uma **região h3**.

**Saída:**
**Matriz de features **contendo em cada linha** a região h3** e em cada coluna a **quantidade de de POIs existentes**.

In [35]:
# checkins_filtrados

In [36]:
# checkins_filtrados[checkins_filtrados['h3_cell'] == '8a44e83598d7fff']

In [37]:
# dados_feature = checkins_filtrados[['h3_cell', 'category', 'placeid']].drop_duplicates(subset=['h3_cell', 'placeid'])

# cross_tab = pd.crosstab(dados_feature['h3_cell'], dados_feature['category'])

# cross_tab

In [38]:
# cross_tab.loc['8a44e83598d7fff']

# PMI de co-ocorrência de localização deslocada (informações mútuas pontuais)

In [39]:
# # retirado do código do HMRM
# def create_location_coocurrency_matrix(checkins):
#         try: 
#             users_checkins_sorted = checkins.sort_values(by=["local_datetime"])
#             regions = users_checkins_sorted["h3_id"].tolist()
#             number_of_regions = checkins["h3_id"].nunique()
            
#             location_co_ocurrency = sparse.lil_matrix(
#                 (number_of_regions, number_of_regions)
#             )  ##location co occurency represents memory for save memory

#             for i in range(len(regions)):
#                 for j in range(1, 6):
#                     if (i - j) < 0:
#                         break
#                     location_co_ocurrency[regions[i], regions[i - j]] += 1
#                 for j in range(1, 6):
#                     if (i + j) > len(regions) - 1:
#                         break
#                     location_co_ocurrency[regions[i], regions[j + i]] += 1
#             sum_of_dl = np.sum(location_co_ocurrency)
#             l_occurrency = np.sum(location_co_ocurrency, axis=1).reshape(-1, 1)
#             c_occurrency = np.sum(location_co_ocurrency, axis=0).reshape(1, -1)

#             for i in range(number_of_regions):
#                 line = location_co_ocurrency[i].toarray()
#                 ##PMI em subdivisoes da matriz esparsa
#                 location_co_ocurrency[i] = np.maximum(
#                     np.log2(
#                         np.maximum(line * sum_of_dl, 1)
#                         / (l_occurrency[i] * c_occurrency)
#                     ),
#                     0,
#                 )
            
#             return (location_co_ocurrency)

#         except Exception as e:
#             raise e

In [40]:
# checkins['h3_id'] = (
#     checkins['h3_cell'].apply(lambda x: np.where(h3_regions == x)[0][0])
# )

# # print(create_location_coocurrency_matrix(checkins))

# PMI de localização e tempo alterado

In [41]:
# def create_region_time_matrix(checkins):
#     regions = checkins["h3_id"].tolist()
#     datetimes = checkins["local_datetime"].tolist()
#     number_of_regions = checkins["h3_id"].nunique()
#     Dt = np.zeros((number_of_regions, 48))

#     for i in range(len(regions)):
#         if datetimes[i].weekday() >= 5:
#             Dt[regions[i]][datetimes[i].hour + 24] += 1
#         else:
#             Dt[regions[i]][datetimes[i].hour] += 1

#     sum_of_dt = np.sum(Dt)
#     l_occurrency = np.sum(Dt, axis=1).reshape(-1, 1)
#     c_occurrency = np.sum(Dt, axis=0).reshape(1, -1)

#     mult = l_occurrency * c_occurrency
#     mult[mult == 0] = -1

#     tmp = np.maximum(Dt * sum_of_dt, 1) / mult
#     tmp[tmp < 0] = 0
#     region_time = np.maximum(np.log2(tmp), 0)

#     return region_time